## Génération des aires - réseau RTE-T 

## Objectif 

Générer les aires de services et les échangeurs associés à partir des données ASFA et de BD CARTO.

Les données ASFA contiennent les aires de service associées à une coordonnée autoroute.
Les données BD CARTO contiennent les représentations graphiques des aires de services (point ou polygone).

Le traitement génère la liste des aires et la liste des échangeurs avec les attributs suivants :

échangeurs :
- `nom_ech` : nom de l'échangeur
- `double` : booléen True si l'échangeur dessert deux aires
- `idx_ech`: identifiant (entier)
- `geometry`: localisation (point)
- `roadname`: nom de la route
- `nature`: 'echangeur aires'

aires : 
- `nom_aire` : nom de l'aire
- `geom_aire` : localisation de l'aire (point)
- `sens_aire` : positionnement relatif à l'échangeur (est, ouest, nord, sud)
- `idx_ech` : index de l'échangeur
- `nature`: 'aire de service'

Trois cas sont distingués :
- l'aire est présente dans les données ASFA mais pas dans les données BD CARTO,
- l'aire est présente dans les données ASFA et dans les données BD CARTO,
- l'aire est présente dans les données BD CARTO mais pas dans les données ASFA.

Si les coordonnées de l'aire ne sont pas connues elles sont calculées à partir de celles de l'échangeur si le sens est connu.
Si les coordonnées de l'échangeur ne sont pas connues, elles sont calculées à partir de celles des aires.

Les aires et stations sont stockés sous la forme de deux fichiers GeoPandas (geojson).

In [1]:
"""import os
import sys
new_path = os.getcwd()[:-26] + 'qualicharge-rtet'
sys.path.append(new_path)"""

"import os\nimport sys\nnew_path = os.getcwd()[:-26] + 'qualicharge-rtet'\nsys.path.append(new_path)"

In [2]:
import json
from shapely import Point
import geopandas as gpd
import pandas as pd
import geo_nx as gnx 
from shapely import MultiPoint, centroid, transform

version = 'V1'
data_path = '../data/'
rtet_path = '../rtet/'
refnat = {'tiles': 'cartodbpositron', 'location': [46.3, 2.3], 'zoom_start': 7}
NATURE = 'nature'
ATT_ECHANGEURS = ['geometry', 'double', 'nom_ech', 'idx_ech', 'roadname', 'nature']
ATT_AIRES = ['geometry', 'nom_aire', 'sens_aire', 'idx_ech', 'nature']

## Données ASFA

Les aires de service sont liées à l'attribut 'nature' (AS).

Codes du champ 'nature':
- BA: barrière
- EC: échangeur
- AR: aire de repos
- AS: aire de service
- BI: bifurcation

In [3]:
def translate(point_sens):
    point, sens = list(point_sens)
    DELTA = 100
    match sens:
        case 'nord':
            return transform(point, lambda x : x + [0, DELTA])
        case 'sud':
            return transform(point, lambda x : x + [0, -DELTA])
        case 'est':
            return transform(point, lambda x : x + [DELTA, 0])
        case 'ouest':
            return transform(point, lambda x : x + [-DELTA, 0])
        case _:
            return point

In [4]:
asfa = pd.read_csv(data_path + "asfa.csv").groupby(['coords', 'type', 'nom']).first().reset_index()
asfa = pd.concat([asfa, pd.json_normalize(asfa['dict'].map(lambda x: str.replace(x, '"', "")).map(lambda x: str.replace(x, "'", '"')).map(json.loads))], axis=1)
#del asfa['Unnamed: 0']
#del asfa['dict']
asfa['geometry'] = asfa['coords'].map(json.loads).map(lambda x: [x[1], x[0]]).map(Point)

aires_as = gpd.GeoDataFrame(asfa.loc[asfa['type']=='AS', ['nom', 'geometry', 'roadname', 'type']], crs='4326').to_crs(2154)
nom_ech =  aires_as['nom'].str.lower().str.split(r"(nord|sud|est|ouest)\Z", n=1, expand=True, regex=True)
aires_as['nature'] = 'aire de service'
aires_as['geom_ech_as'] = aires_as['geometry']
aires_as['nom_ech'] = nom_ech[0]
aires_as['sens_aire'] = nom_ech[1]
aires_as['nom_aire'] = aires_as['nom']
aires_as['geom_aire_as'] = aires_as[['geom_ech_as', 'sens_aire']].apply(translate, axis=1) 
aires_as['geometry'] = aires_as['geom_aire_as']

In [5]:
nb_ech = aires_as[['geom_ech_as', 'nom']].groupby('geom_ech_as').count().reset_index()
nb_ech['double_as'] = nb_ech['nom'] == 2
aires_as = aires_as.merge(nb_ech[['geom_ech_as', 'double_as']], on='geom_ech_as')
aires_as

,nom,geometry,roadname,type,nature,geom_ech_as,nom_ech,sens_aire,nom_aire,geom_aire_as,double_as
0,Contr&ocirc;le Frontalier Nord,POINT (688635.61 6152636.878),A9,AS,aire de service,POINT (688635.61 6152536.878),contr&ocirc;le frontalier,nord,Contr&ocirc;le Frontalier Nord,POINT (688635.61 6152636.878),False
1,Contr&ocirc;le Frontalier Sud,POINT (688322.311 6152704.426),A9,AS,aire de service,POINT (688322.311 6152804.426),contr&ocirc;le frontalier,sud,Contr&ocirc;le Frontalier Sud,POINT (688322.311 6152704.426),False
2,Village Catalan Est,POINT (687365.402 6164397.731),A9,AS,aire de service,POINT (687265.402 6164397.731),village catalan,est,Village Catalan Est,POINT (687365.402 6164397.731),True
3,Village Catalan Ouest,POINT (687165.402 6164397.731),A9,AS,aire de service,POINT (687265.402 6164397.731),village catalan,ouest,Village Catalan Ouest,POINT (687165.402 6164397.731),True
4,La Palme Est,POINT (697714.016 6205657.787),A9,AS,aire de service,POINT (697614.016 6205657.787),la palme,est,La Palme Est,POINT (697714.016 6205657.787),True
...,...,...,...,...,...,...,...,...,...,...,...
371,Angres,POINT (681756.322 7034130.71),A26,AS,aire de service,POINT (681756.322 7034130.71),angres,None,Angres,POINT (681756.322 7034130.71),True
372,Souchez,POINT (681756.322 7034130.71),A26,AS,aire de service,POINT (681756.322 7034130.71),souchez,None,Souchez,POINT (681756.322 7034130.71),True
373,Rely,POINT (655949.185 7052968.245),A26,AS,aire de service,POINT (655949.185 7052968.245),rely,None,Rely,POINT (655949.185 7052968.245),True
374,Saint-Hilaires-Cottes,POINT (655949.185 7052968.245),A26,AS,aire de service,POINT (655949.185 7052968.245),saint-hilaires-cottes,None,Saint-Hilaires-Cottes,POINT (655949.185 7052968.245),True


## données BD CARTO

Les données sont disponibles par région (uniquement PACA ici)

In [6]:
def milieu(list_point):    
    return centroid(MultiPoint([point for point in list_point if point]))
def double(list_point):
    return not None in list(list_point)

In [7]:
aires_ca = gpd.read_file(data_path + "BDCARTO/noeuds_aires.geojson").to_crs(2154)
aires_ca['geometry'] = aires_ca['geometry'].centroid
aires_ca['geom_aire_ca'] = aires_ca['geometry']
gs_carto = gnx.from_geopandas_nodelist(aires_ca, node_attr=True)

aires_ca = aires_ca[['geometry', 'geom_aire_ca']]
aires_doubles = aires_ca.sjoin_nearest(aires_ca, how='left', max_distance=1000, exclusive=True)
aires_ca['geom_ech_ca'] = aires_doubles[['geom_aire_ca_left', 'geom_aire_ca_right']].apply(milieu, axis=1) 
aires_ca['double_ca'] = aires_doubles[['geom_aire_ca_left', 'geom_aire_ca_right']].apply(double, axis=1) 

## Association des données ASFA et BD CARTO

In [8]:
first = aires_ca.groupby('geom_ech_ca').first()[['geom_aire_ca', 'geometry']]
aires_ca['first'] = aires_ca['geom_aire_ca'].isin(first['geom_aire_ca'])

aires_ca_first = aires_ca.loc[aires_ca['first'], :]
aires_ca_second = aires_ca.loc[~aires_ca['first'], :]

In [9]:
aires_as['geometry'] = aires_as['geom_aire_as']
aires_ca_as_first = aires_ca_first.sjoin_nearest(aires_as, how='left', max_distance=1000)
aires_ca_as_first = aires_ca_as_first.groupby(['geometry']).first().reset_index()

aires_as['first'] =  aires_as['geom_aire_as'].isin(aires_ca_as_first['geom_aire_as'])
aires_as_second = aires_as.loc[~aires_as['first'], :]
aires_ca_as_second =  aires_ca_second.sjoin_nearest(aires_as_second, how='left', max_distance=1000).to_crs(aires_ca_as_first.crs)

aires_ca_as = pd.concat([aires_ca_as_first, aires_ca_as_second])

C:\Opensource\anaconda3\envs\afir\Lib\site-packages\geopandas\array.py:1638: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as RGF93 v1 / Lambert-93 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))


In [10]:
aires_ca_not_as = aires_ca_as.loc[pd.isna(aires_ca_as.index_right), :]
aires_ca_and_as = aires_ca_as.loc[~pd.isna(aires_ca_as.index_right), :]

In [11]:
id_aires_as_not_ca = list(set(aires_as.index)-set(aires_ca_as['index_right'].dropna().astype('int')))
aires_as_not_ca = aires_as.loc[id_aires_as_not_ca]

## Génération des fichiers communs

In [12]:
att_communs = ['geometry', 'geom_aire', 'geom_ech', 'double', 'nom_ech', 'nom_aire', 'sens_aire', 'roadname']

In [13]:
com_aires_ca_and_as = aires_ca_and_as.copy()
com_aires_ca_and_as['geometry'] = com_aires_ca_and_as['geom_aire_ca']
com_aires_ca_and_as['geom_aire'] = com_aires_ca_and_as['geom_aire_ca']
com_aires_ca_and_as['geom_ech'] = com_aires_ca_and_as['geom_ech_as']
com_aires_ca_and_as['double'] = com_aires_ca_and_as[['double_as', 'double_ca']].any(axis='columns')
com_aires_ca_and_as = com_aires_ca_and_as[att_communs]

In [14]:
com_aires_ca_not_as = aires_ca_not_as.copy()
com_aires_ca_not_as['geometry'] = com_aires_ca_not_as['geom_aire_ca']
com_aires_ca_not_as['geom_aire'] = com_aires_ca_not_as['geom_aire_ca']
com_aires_ca_not_as['geom_ech'] = com_aires_ca_not_as['geom_ech_ca']
com_aires_ca_not_as['double'] = com_aires_ca_not_as['double_ca']
com_aires_ca_not_as = com_aires_ca_not_as[att_communs]

In [15]:
com_aires_as_not_ca = aires_as_not_ca.copy()
com_aires_as_not_ca['geom_aire'] = com_aires_as_not_ca['geom_aire_as']
com_aires_as_not_ca['geometry'] = com_aires_as_not_ca['geom_aire']
com_aires_as_not_ca['geom_ech'] = com_aires_as_not_ca['geom_ech_as']
com_aires_as_not_ca['double'] = com_aires_as_not_ca['double_as']
com_aires_as_not_ca = com_aires_as_not_ca[att_communs]

In [16]:
com_aires = pd.concat([com_aires_as_not_ca, com_aires_ca_and_as, com_aires_ca_not_as])
com_aires[NATURE] = 'aire de service'
com_ech = com_aires.groupby('geom_ech').first()[['nom_ech', 'roadname', 'double']].reset_index()
com_ech['geometry'] = com_ech['geom_ech']
com_ech[NATURE] = 'echangeur aire'

C:\Opensource\anaconda3\envs\afir\Lib\site-packages\geopandas\array.py:1638: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as RGF93 v1 / Lambert-93 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))


In [17]:
com_ech = gpd.GeoDataFrame(com_ech, crs=2154).reset_index()
com_ech['idx_ech'] = com_ech['index']
com_aires = com_aires.merge(com_ech[['geom_ech', 'idx_ech']], on='geom_ech')
com_ech[ATT_ECHANGEURS].to_file(rtet_path + version + '_echangeurs_aires.geojson', driver='GeoJSON')
com_aires[ATT_AIRES].to_file(rtet_path + version + '_aires_de_service.geojson', driver='GeoJSON')

## Restitution

In [18]:
gs_aires_as_not_ca = gnx.from_geopandas_nodelist(com_aires_as_not_ca, node_attr=True)
gs_aires_ca_and_as = gnx.from_geopandas_nodelist(com_aires_ca_and_as, node_attr=True)
gs_aires_ca_not_as = gnx.from_geopandas_nodelist(com_aires_ca_not_as, node_attr=True)
gs_ech = gnx.from_geopandas_nodelist(com_ech, node_attr=True)

In [19]:
param_exp_nd = {'n_marker_kwds': {'radius': 3, 'fill': True}}
param_exp_gr = {'e_name': 'edges', 'n_name': 'nodes', 'e_popup': ['weight', 'source', 'target'], 'e_tooltip': ["source", "target"], 
                'n_tooltip': ["node_id", "nature"], 'n_marker_kwds': {'radius': 1, 'fill': False}}
carte = gs_aires_as_not_ca.explore(refmap=refnat, n_color='purple', n_name='as_not_ca', edges=False, **param_exp_nd)
carte = gs_aires_ca_and_as.explore(refmap=carte, n_color='green', n_name='ca_and_as', edges=False, **param_exp_nd)
carte = gs_aires_ca_not_as.explore(refmap=carte, n_color='orange', n_name='ca_not_as', edges=False, **param_exp_nd)
carte = gs_ech.explore(refmap=carte, n_color='red', n_name='echangeurs', edges=False, layercontrol=True, **param_exp_nd)
#carte.save('aires_de_service.html')
carte